In [4]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.sparse as sps
from tqdm import tqdm
import gc

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

In [6]:
candidate_items = pd.read_csv("../../Dataset/candidate_items.csv")
item_features = pd.read_csv("../../Dataset/item_features.csv")
train_purchases = pd.read_csv("../../Dataset/train_purchases.csv")
train_sessions = pd.read_csv("../../Dataset/train_sessions.csv")
test_final_sessions = pd.read_csv("../../Dataset/test_final_sessions.csv")
test_leaderboard_sessions = pd.read_csv("../../Dataset/test_leaderboard_sessions.csv")

In [7]:
from RecSys_Course_AT_PoliMi.Pipeline.data_splitting import split_dataframes_val
from RecSys_Course_AT_PoliMi.Pipeline.matrices_creation import create_csr_matrix

weighted_train_set_df, val_purch_df, val_views_df, train_session_mapping, \
val_session_mapping, item_mapping, mapped_items_to_ignore, mapped_val_sessions_arr = split_dataframes_val(
    train_sessions, train_purchases, item_features,
    val_start_ts='2021-05-01', val_end_ts='2021-06-01',
    unique_interactions=True, view_weight=1, purch_weight=1, abs_half_life=None,
    cyclic_decay=False, score_graph=False
)

URM_train = create_csr_matrix(weighted_train_set_df, len(train_session_mapping), len(item_mapping))
URM_val_views = create_csr_matrix(val_views_df, len(val_session_mapping), len(item_mapping))
URM_val_purch = create_csr_matrix(val_purch_df, len(val_session_mapping), len(item_mapping))

In [18]:
list(train_session_mapping.values())[-100:]

[918282,
 918283,
 918284,
 918285,
 918286,
 918287,
 918288,
 918289,
 918290,
 918291,
 918292,
 918293,
 918294,
 918295,
 918296,
 918297,
 918298,
 918299,
 918300,
 918301,
 918302,
 918303,
 918304,
 918305,
 918306,
 918307,
 918308,
 918309,
 918310,
 918311,
 918312,
 918313,
 918314,
 918315,
 918316,
 918317,
 918318,
 918319,
 918320,
 918321,
 918322,
 918323,
 918324,
 918325,
 918326,
 918327,
 918328,
 918329,
 918330,
 918331,
 918332,
 918333,
 918334,
 918335,
 918336,
 918337,
 918338,
 918339,
 918340,
 918341,
 918342,
 918343,
 918344,
 918345,
 918346,
 918347,
 918348,
 918349,
 918350,
 918351,
 918352,
 918353,
 918354,
 918355,
 918356,
 918357,
 918358,
 918359,
 918360,
 918361,
 918362,
 918363,
 918364,
 918365,
 918366,
 918367,
 918368,
 918369,
 918370,
 918371,
 918372,
 918373,
 918374,
 918375,
 918376,
 918377,
 918378,
 918379,
 918380,
 918381]

In [87]:
item_values = item_features.drop(columns=['feature_category_id'])
item_values = item_values.drop_duplicates()
unique_ids = item_values.feature_value_id.unique()
mapping_feature = {k:v for k,v in zip(unique_ids, np.arange(len(unique_ids)))}
item_values['item_id'] = item_values['item_id'].map(item_mapping)
item_values['feature_value_id'] = item_values['feature_value_id'].map(mapping_feature)

ICM_feature_values = sps.csr_matrix((np.ones(shape=(len(item_values,))), (item_values.item_id, item_values.feature_value_id)))

In [11]:
feature_df = item_features.copy()
feature_df = feature_df.sort_values(['item_id', 'feature_category_id', 'feature_value_id'])

# create new column pair feature_category_id-feature_value_id
feature_df['category_value'] = feature_df['feature_category_id'].astype(str) + '-' + feature_df[
    'feature_value_id'].astype(str)
feature_df = feature_df.drop(['feature_value_id', 'feature_category_id'], axis=1).reset_index(drop=True)

# create new index for every pair feature_category_id-feature_value_id
category_value_keys = list(feature_df['category_value'].unique())  # list of pairs category-value
category_value_vals = [v for v in range(len(category_value_keys))]  # new index -- perché non fare arange?
category_value_dict = dict(zip(category_value_keys, category_value_vals))  # create dictionary
category_value_id = [category_value_dict[idx] for idx in
                        feature_df['category_value'].values]  # list of id category-value for the new column

feature_df['category_value_id'] = category_value_id  # A cosa serve modificare il df e aggiungere questa colonna?

feature_df['interaction'] = bool(1)  # perché non fare semplicemente un array di len(feature_df) elementi True?

# reindexing of item_id
mapped_item_id = [item_mapping[idx] for idx in feature_df['item_id'].values]

ICM_all = sps.csr_matrix((feature_df['interaction'].values, (mapped_item_id, category_value_id)))
# Remove rows and columns with only zeros
ICM_all = ICM_all[ICM_all.getnnz(1) > 0][:, ICM_all.getnnz(0) > 0]

In [85]:
feature_df = item_features.copy()
feature_df = feature_df.sort_values(['item_id', 'feature_category_id', 'feature_value_id'])
feature_df['category_value'] = feature_df['feature_category_id'].astype(str) + '-' + feature_df[
        'feature_value_id'].astype(str)
feature_df = feature_df.drop(['feature_value_id', 'feature_category_id'], axis=1).reset_index(drop=True)
category_ids = feature_df.category_value.unique()
mapping_feature = dict(zip(category_ids, np.arange(len(category_ids))))

item_ids = [item_mapping[idx] for idx in feature_df.item_id.values]
cat_ids = [mapping_feature[idx] for idx in feature_df.category_value.values]

ICM_feature_values = sps.csr_matrix((np.ones(shape=(len(feature_df,))), (item_ids, cat_ids)))

In [86]:
(ICM_all!=ICM_feature_values).nnz==0 

True

In [12]:
def create_ICM(df, item_mapping, which="category_value"):
    feature_df = item_features.copy()
    feature_df = feature_df.sort_values(['item_id', 'feature_category_id', 'feature_value_id'])

    if which == "category_value":
        feature_df['category_value'] = feature_df['feature_category_id'].astype(str) + '-' + feature_df['feature_value_id'].astype(str)
        feature_df = feature_df.drop(['feature_value_id', 'feature_category_id'], axis=1).reset_index(drop=True)
        category_ids = feature_df.category_value.unique()
        mapping_feature = dict(zip(category_ids, np.arange(len(category_ids))))

        item_ids = [item_mapping[idx] for idx in feature_df.item_id.values]
        cat_ids = [mapping_feature[idx] for idx in feature_df.category_value.values]

        ICM = sps.csr_matrix((np.ones(shape=(len(feature_df,))), (item_ids, cat_ids)))
    elif which == "feature_value":
        feature_df = feature_df.drop(columns=['feature_category_id'])
        feature_df = feature_df.drop_duplicates()
        unique_ids = feature_df.feature_value_id.unique()
        mapping_feature = {k:v for k,v in zip(unique_ids, np.arange(len(unique_ids)))}
        
        item_ids = [item_mapping[idx] for idx in feature_df.item_id.values]
        cat_ids = [mapping_feature[idx] for idx in feature_df.feature_value_id.values]

        ICM = sps.csr_matrix((np.ones(shape=(len(feature_df,))), (item_ids, cat_ids)))

    else:
        raise NotImplementedError

    return ICM, mapping_feature

In [13]:
# Check if equal

def create_ICM_old(df, item_mapping):
    """
    Return sparse ICM from item_features dataframe and dictionary {'feature_category_id-feature_value_id': new_category_value_idx}

    """
    feature_df = df.copy()
    feature_df = feature_df.sort_values(['item_id', 'feature_category_id', 'feature_value_id'])

    # create new column pair feature_category_id-feature_value_id
    feature_df['category_value'] = feature_df['feature_category_id'].astype(str) + '-' + feature_df[
        'feature_value_id'].astype(str)
    feature_df = feature_df.drop(['feature_value_id', 'feature_category_id'], axis=1).reset_index(drop=True)

    # create new index for every pair feature_category_id-feature_value_id
    category_value_keys = list(feature_df['category_value'].unique())  # list of pairs category-value
    category_value_vals = [v for v in range(len(category_value_keys))]  # new index -- perché non fare arange?
    category_value_dict = dict(zip(category_value_keys, category_value_vals))  # create dictionary
    category_value_id = [category_value_dict[idx] for idx in
                         feature_df['category_value'].values]  # list of id category-value for the new column

    feature_df['category_value_id'] = category_value_id  # A cosa serve modificare il df e aggiungere questa colonna?
    feature_df['interaction'] = bool(1)  # perché non fare semplicemente un array di len(feature_df) elementi True?

    # reindexing of item_id
    mapped_item_id = [item_mapping[idx] for idx in feature_df['item_id'].values]

    ICM_all = sps.csr_matrix((feature_df['interaction'].values, (mapped_item_id, category_value_id)))
    # Remove rows and columns with only zeros
    ICM_all = ICM_all[ICM_all.getnnz(1) > 0][:, ICM_all.getnnz(0) > 0]

    return ICM_all, category_value_dict

In [94]:
# Check if works
(create_ICM(item_features, item_mapping)[0]!=create_ICM_old(item_features, item_mapping)[0]).nnz==0 

True

In [15]:
# Compare performance 

from RecSys_Course_AT_PoliMi.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout
from RecSys_Course_AT_PoliMi.Pipeline.utils import get_cold_sessions

best_hyp = {'topK': 9627, 'shrink': 5404, 'similarity': 'tanimoto', 'feature_weighting': 'none', 'normalize': True}
itemknn = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=create_ICM(item_features, item_mapping, which="feature_value")[0])
itemknn.fit(**best_hyp)
itemknn.set_URM_train(URM_val_views)
_, ignore_users = get_cold_sessions(weighted_train_set_df, val_views_df, item_mapping, item_features)
eval_obj = EvaluatorHoldout(URM_val_purch, [100], ignore_items=mapped_items_to_ignore, ignore_users=ignore_users)
eval_obj.evaluateRecommender(itemknn)


ItemKNNCBFRecommender: URM Detected 477 ( 2.0%) items with no interactions.
Similarity column 23691 (100.0%), 885.67 column/sec. Elapsed time 26.75 sec
EvaluatorHoldout: Ignoring 18922 Items
EvaluatorHoldout: Ignoring 77191 Users
EvaluatorHoldout: Processed 4427 (100.0%) in 5.75 sec. Users per second: 770


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                     
 100     0.004299                 0.429862  0.429862  0.00132    0.131997   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 100     0.131997  0.190302  0.008512  0.429862      0.131997  ...   
 
        COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
 cutoff                                                             
 100                 0.130426           1.0              0.429862   
 
        DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
 cutoff                                                             
 100          0.389709       11.274108                   1.000348   
 
        RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY  \
 cutoff                                                   